In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("python spark sql basic")
    .enableHiveSupport()
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/30 21:45:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [51]:
df = spark.read.json("./user.json")

In [10]:
df.printSchema()

root
 |-- completed: boolean (nullable = true)
 |-- id: long (nullable = true)
 |-- title: string (nullable = true)
 |-- userId: long (nullable = true)



In [22]:
df.groupBy("userId").length("title").show()

AttributeError: 'GroupedData' object has no attribute 'length'

In [30]:
df.createOrReplaceTempView("todos")
sqlDf = spark.sql("select * from todos where todos.userId > 5 order by todos.userId desc")
sqlDf.show()

+---------+---+--------------------+------+
|completed| id|               title|userId|
+---------+---+--------------------+------+
|    false|184|molestias modi pe...|    10|
|    false|181|ut cupiditate seq...|    10|
|     true|182|inventore saepe c...|    10|
|     true|183|omnis nulla eum a...|    10|
|    false|185|voluptates dignis...|    10|
|    false|186|explicabo odio es...|    10|
|    false|187|consequuntur anim...|    10|
|     true|188|  vel non beatae est|    10|
|     true|189|culpa eius et vol...|    10|
|     true|190|accusamus sint iu...|    10|
|     true|191|temporibus atque ...|    10|
|    false|192|ut quas possimus ...|    10|
|     true|193|rerum debitis vol...|    10|
|    false|194|sed ut vero sit m...|    10|
|     true|195|rerum ex veniam m...|    10|
|     true|196|consequuntur aut ...|    10|
|     true|197|dignissimos quo n...|    10|
|     true|198|quis eius est sin...|    10|
|     true|199|numquam repellend...|    10|
|    false|200|ipsam aperiam vol

In [35]:
from pyspark.sql.types import StringType, StructType, StructField

sc = spark.sparkContext
lines = sc.textFile("people.txt")
parts = lines.map(lambda l: l.split(","))
people = parts.map(lambda p: (p[0], p[1].strip()))
schemaString = "name age"
fields = [
    StructField(field_name, StringType(), True) for field_name in schemaString.split()
]
schema = StructType(fields)
schemaPeople = spark.createDataFrame(people, schema)
schemaPeople.createOrReplaceTempView("people")

In [48]:
df = spark.read.load("people.txt", format="csv", sep=",", inferSchema=True)

In [79]:
df.write.partitionBy("userId").saveAsTable("t")

In [82]:
spark.sql("show partitions t").show()

+---------+
|partition|
+---------+
| userId=1|
|userId=10|
| userId=2|
| userId=3|
| userId=4|
| userId=5|
| userId=6|
| userId=7|
| userId=8|
| userId=9|
+---------+



In [2]:
spark.sql("CREATE TABLE IF NOT EXISTS src (key INT, value STRING) USING hive")

23/06/30 21:45:45 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/06/30 21:45:45 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
23/06/30 21:46:26 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
23/06/30 21:46:26 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore srivatsan@192.168.1.33
23/06/30 21:46:26 WARN ObjectStore: Failed to get database default, returning NoSuchObjectException
23/06/30 21:46:30 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
23/06/30 21:46:30 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
23/06/30 21:46:30 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/06/30 21:46:30 W

DataFrame[]